<a href="https://colab.research.google.com/github/MohsenJadidi/Automatic-Modulation-Classification-AMC/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Importing the dataset (Copy from CNN.ipynb and merge cells)
without Reshape(2 end line)

In [2]:
import pickle
import numpy as np
fileName = 'RML2016.10a_dict.pkl'
with open("/content/drive/My Drive/Colab Notebooks/"+fileName,'rb') as f:
  data = pickle.load(f,encoding='bytes')
 
X = []
labels = [] # label each example by a pair (modulation type, snr)
total_examples = 0
analog = [b'AM-DSB', b'AM-SSB', b'WBFM']

for mod_type, snr in data.keys():
    if (mod_type not in analog):      
        current_matrix = data[(mod_type, snr)]        
        total_examples += current_matrix.shape[0]
        for i in range(current_matrix.shape[0]):
            X.append(current_matrix[i])
            labels.append((str(mod_type, 'ascii'), snr)) # mod_type is of type bytes
    
X = np.array(X)         # First row is QPSK snr=2, seconde is PAM4 snr=8 , ...
labels = np.array(labels)

y = labels[:,0]

print(f'loaded {total_examples} signal vectors into X{X.shape} and their corresponding'
      f' labels into labels{labels.shape}')  
# print(np.unique(labels[:,0]))

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
onehotencoder = OneHotEncoder()
y = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()

snrList = [str(2*i-20) for i in range(20)]  # snrList = -20, -18, -16 , ... ,0, ... ,18
snr = snrList[19]
numberOfEachExamples = 1000
print("SNR :", snr)

output = [[labels[i*numberOfEachExamples, 0],y[i*numberOfEachExamples]] for i in range(int(X.shape[0]/numberOfEachExamples))]
output = dict(output)

Xsnr = np.zeros(shape=(X.shape[0],X.shape[1],X.shape[2]+1))
for i in range(X.shape[0]):
    snr = int(labels[i,1])
    Xsnr[i,0,:] = np.insert(X[i,0,:],0,snr)
    Xsnr[i,1,:] = np.insert(X[i,1,:],0,snr)
    
print(Xsnr.shape)

###### Splitting the dataset into the Training set and Test set ######
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xsnr, y, test_size = 0.2, random_state = 0)
# The below line better for Cross_val part
#X_train, X_test, y_train, y_test = train_test_split(Xsnr, y, test_size = 1, random_state = 0)

print(X_train.shape)
print(X_test.shape)


X_train = X_train[:,:,1:] # snr important for train

y_test18 = []
X_test18 = []

for i in range(X_test.shape[0]):
    if X_test[i,0,0] == 18:
        X_test18.append(X_test[i])
        y_test18.append(y_test[i])
        
X_test18 = np.array(X_test18)
y_test18 = np.array(y_test18)        
X_test18 = X_test18[:,:,1:]


print(X_train.shape)
print(X_test18.shape)


# Change IQ to amplitude and phase
X_cmplx = X_train[:,0,:] + 1j* X_train[:,1,:]    
X_amp = np.abs(X_cmplx)
X_amp = X_amp/(np.reshape(np.sqrt(np.mean(X_amp**2,axis=1)),(-1,1))*np.ones([1,128]))
X_ang = np.arctan2(X_train[:,1,:],X_train[:,0,:]) / np.pi
    
X_amp = np.reshape(X_amp,(-1,1,128))
X_ang = np.reshape(X_ang,(-1,1,128))
    
X_train_AmpPhs = np.concatenate((X_amp,X_ang), axis=1) 
##
X_cmplx = X_test18[:,0,:] + 1j* X_test18[:,1,:]    
X_amp = np.abs(X_cmplx)
X_ang = np.arctan2(X_test18[:,1,:],X_test18[:,0,:]) / np.pi
    
X_amp = np.reshape(X_amp,(-1,1,128))
X_ang = np.reshape(X_ang,(-1,1,128))
    
X_test18_AmpPhs = np.concatenate((X_amp,X_ang), axis=1) 
##

print(X_train_AmpPhs.shape)
print(X_test18_AmpPhs.shape)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = X_train.reshape([X_train.shape[0],256])
#X_train = X_train_AmpPhs.reshape([X_train.shape[0],256])
X_train = sc.fit_transform(X_train)
X_train = X_train.reshape([X_train.shape[0],2,128])
#X_test = X_test.reshape([1600,256])
X_test18 = X_test18.reshape([X_test18.shape[0],256])
#X_test18 = X_test18_AmpPhs.reshape([X_test18.shape[0],256])
X_test18 = sc.transform(X_test18)
X_test18 = X_test18.reshape([X_test18.shape[0],2,128])

# Reshape
X_train = X_train.reshape(-1,2, 128, 1)   #Reshape for CNN -  (6400,2,128)->(6400,2,128,1)!!
X_test18 = X_test18.reshape(-1,2, 128, 1)

print(X_train.shape)
print(X_test18.shape)

loaded 160000 signal vectors into X(160000, 2, 128) and their corresponding labels into labels(160000, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


SNR : 18
(160000, 2, 129)
(128000, 2, 129)
(32000, 2, 129)
(128000, 2, 128)
(1653, 2, 128)
(128000, 2, 128)
(1653, 2, 128)
(128000, 2, 128, 1)
(1653, 2, 128, 1)


## Import Module

In [3]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import add
from keras.layers import Reshape
from keras.layers import SimpleRNN, LSTM
from keras.layers import BatchNormalization, SpatialDropout1D
from keras.layers.convolutional import ZeroPadding2D


Using TensorFlow backend.


In [0]:
# For Simple RNN
X_train = X_train.reshape(-1,2, 128)
X_test18 = X_test18.reshape(-1,2, 128)

In [0]:
classifier = Sequential()


dout = 0.25
classifier.add(SimpleRNN(256, input_shape =  (2,128), activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(SimpleRNN(256, activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(SimpleRNN(256, activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(SimpleRNN(256, activation = 'relu'))
classifier.add(Dropout(dout))



#classifier.add(Flatten())
#classifier.add(Dense(output_dim = 128 , activation = 'relu')) # hidden layer
#classifier.add(Dropout(dout))
classifier.add(Dense(output_dim = 8, activation = 'softmax'))

adamOpt = keras.optimizers.adam(lr=0.001)

classifier.compile(optimizer = adamOpt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_25 (SimpleRNN)    (None, 256)               98560     
_________________________________________________________________
dropout_43 (Dropout)         (None, 256)               0         
_________________________________________________________________
reshape_14 (Reshape)         (None, 1, 256)            0         
_________________________________________________________________
simple_rnn_26 (SimpleRNN)    (None, 256)               131328    
_________________________________________________________________
dropout_44 (Dropout)         (None, 256)               0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 1, 256)            0         
_________________________________________________________________
simple_rnn_27 (SimpleRNN)    (None, 256)             

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`


#Making LSTM0 model
(Thesis :DEEP NEURAL NETWORK ARCHITECTURES FOR MODULATION CLASSIFICATION)

In [5]:
classifier = Sequential()


dout = 0.25
classifier.add(LSTM(128, input_shape =  (2,128), activation = 'relu' ,return_sequences=True))
classifier.add(Dropout(dout))
#classifier.add(Reshape(target_shape = (1,128)))
classifier.add(LSTM(128, activation = 'relu'))
classifier.add(Dropout(dout))

'''
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(LSTM(256, activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(LSTM(256, activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,256)))
classifier.add(LSTM(256, activation = 'relu'))
classifier.add(Dropout(dout))
'''



#classifier.add(Flatten())
#classifier.add(Dense(output_dim = 128 , activation = 'relu')) # hidden layer
#classifier.add(Dropout(dout))

classifier.add(Dense(output_dim = 8, activation = 'softmax'))

adamOpt = keras.optimizers.adam(lr=0.001)

classifier.compile(optimizer = adamOpt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2, 256)            394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 256)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`


#Making LSTM1 model

In [0]:
classifier = Sequential()


dout = 0.7
classifier.add(LSTM(128, input_shape =  (2,128), activation = 'tanh', return_sequences=True, recurrent_dropout=0.2))
classifier.add(SpatialDropout1D(0.5))
#classifier.add(BatchNormalization())
#classifier.add(Dropout(dout))
# classifier.add(Reshape(target_shape = (1,128)))
classifier.add(LSTM(128, activation = 'tanh', return_sequences=True, recurrent_dropout=0.2))
classifier.add(SpatialDropout1D(0.5))
#classifier.add(BatchNormalization())
#classifier.add(Dropout(dout))


classifier.add(Flatten())
#classifier.add(BatchNormalization())
#classifier.add(Dense(output_dim = 128 , activation = 'relu')) # hidden layer
#classifier.add(Dropout(dout))

classifier.add(Dense(output_dim = 8, activation = 'softmax'))

adamOpt = keras.optimizers.adam(lr=0.001)

classifier.compile(optimizer = adamOpt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")

# Fitting model

In [6]:
batch = 1024
epoch = 100

history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch, validation_data=(X_test18, y_test18))
#history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 128000 samples, validate on 1653 samples
Epoch 1/100
128000/128000 [==============================] - 15s 119us/step - loss: 1.9696 - acc: 0.1832 - val_loss: 1.5870 - val_acc: 0.3448
Epoch 2/100
128000/128000 [==============================] - 7s 55us/step - loss: 1.6816 - acc: 0.3095 - val_loss: 1.1861 - val_acc: 0.4822
Epoch 3/100
128000/128000 [==============================] - 7s 55us/step - loss: 1.5304 - acc: 0.3619 - val_loss: 1.0925 - val_acc: 0.5003
Epoch 4/100
128000/128000 [==============================] - 7s 55us/step - loss: 1.4703 - acc: 0.3836 - val_loss: 1.0092 - val_acc: 0.5306
Epoch 5/100
128000/128000 [==============================] - 7s 55us/step - loss: 1.4242 - acc: 0.3954 - val_loss: 0.9753 - val_acc: 0.5209
Epoch 6/100
128000/128000 [==============================] - 7s 54us/step - loss: 1.3994 - acc: 0.4038 - val_loss: 0.9538 - val_acc: 0.5330
Epoch 7/100
12

# Save model

In [7]:
dic = "/content/drive/My Drive/Colab Notebooks/"
classifier.save(dic+f'SaveModel/LSTM-256-256-256-256-256-ba{batch}-ep{epoch}-(dout=0.25)-input(IQ)(SNR=all).h5')
f = open(dic+f'SaveModel/LSTM-256-256-256-256-256-ba{batch}-ep{epoch}-(dout=0.25)-input(IQ)(SNR=all)-history.txt',"w")
f.write( str(classifier.history.history) )
f.close()

print('Model Saved!')

Model Saved!



# Prediction(only SNR=18)

In [0]:
y_pred = classifier.predict(X_test18)
y_pred = np.argmax(y_pred, axis=1)

y_real = np.argmax(y_test18, axis=1)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_real, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

acc_test = classifier.evaluate(X_test18, y_test18)[1]
acc_train = classifier.evaluate(X_train, y_train)[1]

print("Acc Test : ", acc_test)
print("Acc Train : ", acc_train)


# Plot Confusion Matrix

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd

modulation_order = []
modulation_order_dict = dict()

for key,value in output.items():
    modulation_order_dict[np.argmax(value)] = str(key)
    
for i in range(8):
    modulation_order.append(modulation_order_dict[i])
    
acc_test = classifier.evaluate(X_test18, y_test18)[1]
acc_train = classifier.evaluate(X_train, y_train)[1]

print("Acc Test : ", acc_test)
print("Acc Train : ", acc_train)    

In [8]:
dic = "/content/drive/My Drive/Colab Notebooks/"
acc = []

for snr in snrList:
  print('SNR: ', snr)
    
  y_test_snr = []
  X_test_snr = []
  for i in range(X_test.shape[0]):
    if X_test[i,0,0] == int(snr):
      X_test_snr.append(X_test[i])
      y_test_snr.append(y_test[i])
        
  X_test_snr = np.array(X_test_snr)
  y_test_snr = np.array(y_test_snr)        
  X_test_snr = X_test_snr[:,:,1:]
  
  X_test_snr = X_test_snr.reshape([X_test_snr.shape[0],256])
  X_test_snr = sc.transform(X_test_snr)
  X_test_snr = X_test_snr.reshape([X_test_snr.shape[0],2,128])
  
 # X_test_snr = X_test_snr.reshape(-1,2, 128, 1) # For CNN
    
  acc_test = classifier.evaluate(X_test_snr, y_test_snr)[1]
  acc.append(acc_test)
  print(acc_test)
  '''
  y_pred = classifier.predict(X_test_snr)
  y_pred = np.argmax(y_pred, axis=1)

  y_real = np.argmax(y_test_snr, axis=1)
  # Making the Confusion Matrix
  cm = confusion_matrix(y_real, y_pred)
  cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
  cmDataFrame = pd.DataFrame(cm_norm, index=modulation_order, columns = modulation_order)
  plt.figure(figsize=(6, 5))
  ax = sns.heatmap(cmDataFrame, annot=True, annot_kws={"size": 8}, fmt='.2f', linewidths=.5, cmap="Blues")

  plt.title(f"CNN0[128(1,7)-64(2,5)] Confusion Matrix (SNR={snr})")
  plt.xlabel("Predicted label  \n\n TrainAcc={:.2}, TestAcc={:.2}".format(acc_train,acc_test), fontsize=8)
  plt.ylabel("True lable", fontsize=8)
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor", fontsize=8)
  plt.setp(ax.get_yticklabels(), fontsize=8)
  fig = ax.get_figure()

  fig.savefig(dic+f"/Pic/CNN/CNN0-128(1,7)-64(2,5)-ba{batch}-ep{epoch}-(dout=0.5)-input(IQ)(SNR={snr}).png", dpi=175, bbox_inches='tight')
  fig.savefig(dic+f"/Pic/CNN/CNN0-128(1,7)-64(2,5)-ba{batch}-ep{epoch}-(dout=0.5)-input(IQ)(SNR={snr}).eps", bbox_inches='tight')
  print("Plot Saved!")
'''

  del(y_test_snr)
  del(X_test_snr)
      
print(acc)    


SNR:  -20
1636/1636 [==============================] - 1s 341us/step
0.132640586797066
SNR:  -18
1630/1630 [==============================] - 1s 356us/step
0.11595092027739513
SNR:  -16
1606/1606 [==============================] - 1s 363us/step
0.136986301369863
SNR:  -14
1633/1633 [==============================] - 1s 349us/step
0.15125535823637476
SNR:  -12
1567/1567 [==============================] - 1s 333us/step
0.14167198455097876
SNR:  -10
1561/1561 [==============================] - 1s 344us/step
0.18001281224253238
SNR:  -8
1605/1605 [==============================] - 1s 380us/step
0.203115264806792
SNR:  -6
1605/1605 [==============================] - 1s 372us/step
0.2579439252429291
SNR:  -4
1600/1600 [==============================] - 1s 343us/step
0.38125
SNR:  -2
1642/1642 [==============================] - 1s 354us/step
0.5456760050173073
SNR:  0
1570/1570 [==============================] - 1s 347us/step
0.6388535031847133
SNR:  2
1577/1577 [=============================